# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")


import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient

str_conn='mongodb://localhost:27017'
client = MongoClient(str_conn)

In [2]:
import pandas as pd

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [3]:
# Your Code
client.list_database_names()

['admin', 'companies', 'config', 'local']

In [48]:
db = client.companies

In [5]:
colec = db.companies

In [7]:
query = {'name': 'Babelgum'}
filtro = {'name': True, '_id': False}
list(colec.find(query, filtro))

[{'name': 'Babelgum'}]

In [11]:
df=pd.DataFrame(list(colec.find(query, filtro)))
df

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [34]:
query={'number_of_employees': {'$gt': 5000}}
filtro = {'name': True, 'number_of_employees' : True, '_id': False}
list(colec.find(query, filtro).sort('number_of_employees', -1).limit(2)) 


[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000}]

In [35]:
df=pd.DataFrame(list(colec.find(query, filtro).sort('number_of_employees', -1).limit(2)))
df 

,name,number_of_employees
0,Siemens,405000
1,IBM,388000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [40]:
query={'$and':[{'founded_year': {'$gte': 2000}}, 
               {'founded_year': {'$lte': 2005}}]}
filtro = {'name': True,'founded_year' : True, '_id': False}
list(colec.find(query, filtro).limit(2))


[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005}]

In [42]:
df=pd.DataFrame(list(colec.find(query, filtro).limit(4)))
df 

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [53]:
query={'$and':[{'ipo.valuation_amount': {'$gte': 1e8}}, 
               {'founded_year': {'$lte': 2010}}]}
filtro = {'name': True,'founded_year' : True, 'ipo.valuation_amount': True, '_id' : False}
raw_tabla = list(colec.find(query, filtro).limit(2))
raw_tabla

[{'name': 'Facebook',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter',
  'founded_year': 2006,
  'ipo': {'valuation_amount': 18100000000}}]

In [89]:
good_tabla = []
for dic in raw_tabla:

    tmp = []
    for v in dic:
        if v == 'ipo':
            tmp.append(dic['ipo']['valuation_amount'])
        else:
            tmp.append(dic[v])
    good_tabla.append(tmp)
        

pd.DataFrame(good_tabla, columns=['name', 'founded_year', 'valuation_amount']) #GrandreAdri #SiguientePartidoMeDejoGanar

,name,founded_year,valuation_amount
0,Facebook,2004,104000000000
1,Twitter,2006,18100000000


In [78]:
raw_tabla[0]['ipo']['valuation_amount']

104000000000

In [52]:
df=pd.DataFrame(list(colec.find(query, filtro).limit(2)))
df 

,name,founded_year,ipo
0,Facebook,2004,{'valuation_amount': 104000000000}
1,Twitter,2006,{'valuation_amount': 18100000000}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [90]:
query={'$and':[{'number_of_employees': {'$lt': 1000}}, 
               {'founded_year': {'$lt': 2005}}]}
filtro = {'name': True,'founded_year' : True, 'number_of_employees': True, '_id' : False}
list(colec.find(query, filtro).sort('number_of_employees', -1).limit(10))


[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Buongiorno', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'Cvent', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000}]

In [93]:
df=pd.DataFrame(list(colec.find(query, filtro).sort('number_of_employees', -1).limit(10)))
df

,name,number_of_employees,founded_year
0,Infinera Corporation,974,2000
1,NorthPoint Communications Group,948,1997
2,888 Holdings,931,1997
3,Forrester Research,903,1983
4,Webmetrics,900,1999
5,SonicWALL,900,1991
6,Cornerstone OnDemand,881,1999
7,Buongiorno,800,1999
8,Cvent,800,1999
9,ZoomInfo,800,2000


### 6. All the companies that don't include the `partners` field.

In [105]:
query={'$and':[{'partners': {'$ne': None}}]}
filtro = {'name': True, '_id' : False}
list(colec.find(query, filtro).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'AdventNet'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Facebook'}]

In [110]:
not_partners=pd.DataFrame(list(colec.find(query, filtro).limit(5)))
not_partners


,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [112]:
query={'$and':[{'category_code': None}]}
filtro = {'name': True, '_id' : False}
list(colec.find(query, filtro).limit(5))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

In [114]:
category_nulls = pd.DataFrame(list(colec.find(query, filtro).limit(5)))
category_nulls

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [123]:
query={'$and':[{'number_of_employees': {'$gte': 100} | {'$lt': 1000}}]}
filtro = {'name': True,'number_of_employees': True, '_id' : False}
list(colec.find(query, filtro).limit(5))


[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

In [124]:
numbemployee = pd.DataFrame(list(colec.find(query, filtro).sort('number_of_employees', -1).limit(5)))
numbemployee

,name,number_of_employees
0,Datamonitor,984
1,Infinera Corporation,974
2,Box,950
3,NorthPoint Communications Group,948
4,888 Holdings,931


### 9. Order all the companies by their IPO price in a descending order.

In [135]:
query = {}
filtro = {'name': True, '_id' : False, 'ipo.valuation_amount' : True}
list(colec.find(query, filtro).sort('ipo.valuation_amount', -1).limit(5))


[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

In [136]:
by_ipo = pd.DataFrame(list(colec.find(query, filtro).sort('ipo.valuation_amount', -1).limit(5)))
by_ipo

,name,ipo
0,GREE,{'valuation_amount': 108960000000}
1,Facebook,{'valuation_amount': 104000000000}
2,Amazon,{'valuation_amount': 100000000000}
3,Twitter,{'valuation_amount': 18100000000}
4,Groupon,{'valuation_amount': 12800000000}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [137]:
query = {}
filtro = {'name': True, '_id' : False, 'number_of_employees' : True}
list(colec.find(query, filtro).sort('number_of_employees', -1).limit(5))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

In [139]:
more_employees = pd.DataFrame(list(colec.find(query, filtro).sort('number_of_employees', -1).limit(5)))
more_employees

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [147]:
query= {'founded_month':{'$gt': 6}}

filtro={'_id': False, 'name': True}

list(colec.find(query, filtro).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'eBay'}]

In [146]:
from_secondsem = pd.DataFrame(list(colec.find(query, filtro).limit(5)))

from_secondsem

,name
0,Wetpaint
1,Zoho
2,Digg
3,Omnidrive
4,eBay


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [153]:
query= {'$and':[{'founded_year':{'$lt':2000}},
                {'acquisition.price_amount':{'$gt': 10000000}}]}

filtro={'_id': False, 'name': True, 'founded_year': True, 'acquisition.price_amount':True}


raw_tablas = list(colec.find(query, filtro).limit(5))

In [154]:
before_2000= pd.DataFrame(list(colec.find(query, filtro).limit(5)))

before_2000

,name,founded_year,acquisition
0,Postini,1999,{'price_amount': 625000000}
1,SideStep,1999,{'price_amount': 180000000}
2,Recipezaar,1999,{'price_amount': 25000000}
3,PayPal,1998,{'price_amount': 1500000000}
4,Snapfish,1999,{'price_amount': 300000000}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [158]:
query= {'acquisition.acquired_year':{'$gt':2010}}

filtro={'_id': False, 'name': True, 'acquisition':True}


list(colec.find(query, filtro).sort('acquisition.price_amount', -1).limit(2))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [160]:
query= {}

filtro={'_id': False, 'name': True, 'founded_year': True}

list(colec.find(query, filtro).sort('founded_year', -1).limit(5))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [164]:
query= {'founded_day':{'$lte':7}}

filtro={'_id': False, 'name': True, 'founded_day':True, 'acquisition.price_amount':True}

list(colec.find(query, filtro).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [165]:
query= {'$and':[{'category_code':'web'},
                {'number_of_employees':{'$gt': 4000}}]}
filtro={'_id': False, 'name': True, 'category_code': True, 'number_of_employees':True}


list(colec.find(query, filtro).sort('number_of_employees', -1).limit(10))

[{'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [166]:
query= {'$and':[{'acquisition.price_amount':{'$gt': 10000000}},
                {'acquisition.price_currency_code': 'EUR'}]}

filtro={'_id': False, 'name': True,'acquisition.price_amount':True, 'acquisition.price_currency_code': True}


list(colec.find(query, filtro).limit(10))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [169]:
query= {'acquisition.acquired_month':{'$lt': 4}}

filtro={'_id': False, 'name': True, 'acquisition': True}


list(colec.find(query, filtro).limit(3))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [172]:
query= {'$and':[{'founded_year':{'$gte': 2000}},
                {'founded_year':{'$lte': 2010}},
                {'acquisition.acquired_year':{'$gt': 2010}}]}
    
filtro={'_id': False, 'name': True,'founded_year':True, 'acquisition.acquired_year': True}


list(colec.find(query, filtro).limit(7))

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'delicious',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2011}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code